In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.display import HTML, display

In [2]:
#Get titles from one page
def titles(section, tag, attr, title_list, link_list):
    for i, h in enumerate(section.findAll(tag, attrs = attr)):
        #print(h['href'])
        link = h.findAll('a', {'class': None})
        if link:
            for a in link:
                link_list.append(a['href'])
                for title in a:
                    title_list.append(title)

#Get dates from one page
def dates(section, tag, attr, date_list):
    for i, h in enumerate(section.findAll(tag, attrs = attr)):
        for date in h:
            date_list.append(date[-10:])

#Collect titles and dates from main page
def collector(soup, tag, attr, title_list, date_list, link_list):
    for i, h in enumerate(soup.findAll(tag, attrs = attr)):
        article = h.findAll('li', {'class':'b-list__item'})
        if article:
            for section in article:
                titles(section, 'h2', {'class':'b-list__item-title'}, title_list, link_list)
                #date_list.append()
                dates(section, 'div', {'class':'b-list__item-date'}, date_list)
            return(article[-1]['data-next']+'?prev_id='+article[-1]['data-id']+'&prev_date='+article[-1]['data-date'])
                
#Collect titles and dates from not main pages
def new_collector(soup, tag, attr, title_list, date_list, link_list):
    for i, h in enumerate(soup.findAll(tag, attrs = attr)):
        for article in h:
            titles(article, 'h2', {'class':'b-list__item-title'}, title_list, link_list)
            dates(article, 'div', {'class':'b-list__item-date'}, date_list)
            #print(article)
            #print('###')
        #meta = soup.findAll('li', attrs = attr)
    return(h['data-next']+'?prev_id='+h['data-id']+'&prev_date='+h['data-date'])


#html-parcer
def soup_fun(section):
    link = 'https://rsport.ria.ru/'
    req = requests.get(link + section)
    soup = BeautifulSoup(req.text, 'lxml')
    return soup

def recursor(title_list, date_list, link_list, link):
    section = 'winter2018_news/'
    print(date_list[-1])
    try:
        new_soup = soup_fun(link)
        new_link = section + new_collector(new_soup, 'li', {'class':'b-list__item'}, title_list, date_list, link_list)
        if date_list[-1] != '01.01.2018' and int(date_list[-1][-4:]) >= 2018:
            recursor(title_list, date_list, link_list, new_link)
    except TimeoutError:
        raise('Oooops, TimeoutError')

In [3]:
title_list = []
date_list = []
link_list = []

section = 'winter2018_news/'
soup = soup_fun(section)
new_link = section + collector(soup, 'ul', {'id':'all', 'class':'b-list__list'}, title_list, date_list, link_list)
recursor(title_list, date_list, link_list, new_link)

print('title_list', len(title_list))
print('date_list', len(date_list))
print('link_list', len(link_list))
print(title_list[-1])

with open('titles_og2018_till_01.01.2017.txt', 'w', encoding = 'utf-8') as f:
    for title, link in zip(title_list, link_list):
        f.write(title + '\nhttps://ria.ru' + link + '\n###\n')

10.03.2018
01.03.2018
27.02.2018
26.02.2018
25.02.2018


KeyboardInterrupt: 